In [2]:
import pickle
import numpy as np

In [3]:
DATA_FILE_PATH = 'smokin2mp4.pkl'
with open(DATA_FILE_PATH, 'rb') as file:
    data = pickle.load(file)

## data format: list of dicts for the analyzed frames of the following fields:
{'frame_id', 'body_parts', 'all_peaks', 'subset', 'candiate'}

In [4]:
COCO_BODY_PARTS = ['nose', 'neck',
                   'right_shoulder', ' right_elbow', 'right_wrist',
                   'left_shoulder', 'left_elbow', 'left_wrist',
                   'right_hip', 'right_knee', 'right_ankle',
                   'left_hip', 'left_knee', 'left_ankle',
                   'right_eye', 'left_eye', 'right_ear', 'left_ear', 'background'
                   ]
len(COCO_BODY_PARTS)

19

## body parts
one example of coordiinates for every COCO body part. USELESS

In [5]:
body_parts = np.array(data[25]['body_parts'])
body_parts

array({'nose': (230, 87), 'neck': (145, 100), 'right_shoulder': (228, 97), ' right_elbow': (127, 114), 'right_wrist': (130, 129), 'left_shoulder': (157, 100), 'left_elbow': (159, 121), 'left_wrist': (145, 133), 'right_hip': (229, 134), 'right_knee': (226, 156), 'right_ankle': (223, 174), 'left_hip': (154, 134), 'left_knee': (153, 160), 'left_ankle': (151, 184), 'right_eye': (229, 84), 'left_eye': (233, 84), 'right_ear': (138, 88), 'left_ear': (241, 85)},
      dtype=object)

## all_peaks
a list of lists each corresponding to a COCO body part made of tuples of the following format of locs:
(x, y, confidence, peak_id)

In [6]:
all_peaks = np.array(data[0]['all_peaks'])
all_peaks

array([list([(234, 87, 0.9083782434463501, 0), (181, 96, 0.5591726899147034, 1)]),
       list([(169, 99, 0.765608012676239, 2), (241, 100, 0.9036915898323059, 3), (196, 118, 0.8418397307395935, 4)]),
       list([(230, 97, 0.864372193813324, 5), (163, 100, 0.8208009600639343, 6), (210, 124, 0.8031818270683289, 7)]),
       list([(227, 114, 0.7221102118492126, 8), (158, 120, 0.8832500576972961, 9), (215, 151, 0.8135343790054321, 10)]),
       list([(178, 115, 0.6068695187568665, 11), (226, 129, 0.6727707982063293, 12), (232, 157, 0.5008898377418518, 13)]),
       list([(177, 99, 0.5348705053329468, 14), (251, 103, 0.8400102257728577, 15), (182, 113, 0.7927486896514893, 16)]),
       list([(252, 124, 0.8243544101715088, 17)]),
       list([(244, 142, 0.7252730131149292, 18)]),
       list([(167, 134, 0.8585205674171448, 19), (230, 134, 0.7712255120277405, 20), (204, 167, 0.680644690990448, 21)]),
       list([(226, 155, 0.8229906558990479, 22), (167, 157, 0.8823679089546204, 23), (201, 

## subset
an array in which each row corresponds to a different "object" with the row items indicating the peak_id's in all_peaks. the column index corresponds to the COCO body part index

    # last number in each row is the total parts number of that person
    # the second last number in each row is the score of the overall configuration

In [7]:
subset = np.array(data[0]['subset'])
subset

array([[ 0.        ,  3.        ,  5.        ,  8.        , 12.        ,
        15.        , 17.        , 18.        , 20.        , 22.        ,
        25.        , 29.        , 32.        , 35.        , 37.        ,
        39.        , -1.        , 43.        , 28.89961997, 17.        ],
       [-1.        ,  4.        ,  7.        , 10.        , 13.        ,
        16.        , -1.        , -1.        , 21.        , 24.        ,
        27.        , 30.        , 33.        , 36.        , -1.        ,
        -1.        , 42.        , 44.        , 18.56997137, 13.        ],
       [ 1.        ,  2.        ,  6.        ,  9.        , 11.        ,
        14.        , -1.        , -1.        , 19.        , 23.        ,
        26.        , 28.        , 31.        , 34.        , 38.        ,
        40.        , 41.        , -1.        , 21.81312792, 15.        ]])

## Candidate
array in which each row is one of the peaks, same as each tuple in all_peaks

In [8]:
candidate = np.array(data[0]['candidate'])
candidate

array([[234.        ,  87.        ,   0.90837824,   0.        ],
       [181.        ,  96.        ,   0.55917269,   1.        ],
       [169.        ,  99.        ,   0.76560801,   2.        ],
       [241.        , 100.        ,   0.90369159,   3.        ],
       [196.        , 118.        ,   0.84183973,   4.        ],
       [230.        ,  97.        ,   0.86437219,   5.        ],
       [163.        , 100.        ,   0.82080096,   6.        ],
       [210.        , 124.        ,   0.80318183,   7.        ],
       [227.        , 114.        ,   0.72211021,   8.        ],
       [158.        , 120.        ,   0.88325006,   9.        ],
       [215.        , 151.        ,   0.81353438,  10.        ],
       [178.        , 115.        ,   0.60686952,  11.        ],
       [226.        , 129.        ,   0.6727708 ,  12.        ],
       [232.        , 157.        ,   0.50088984,  13.        ],
       [177.        ,  99.        ,   0.53487051,  14.        ],
       [251.        , 103

# Create data structure per person in video
Try to create a 3D array of coordinates and time for each person

1) parse pkl for coordinates for each body part per frame

2) match same person between frames - based on the minimum change in head location between consecutive frames

3) filter in time out bad pose estimates, based on:
    a. smoothing filter
    b. pose estimate confidence
 

## How many persons there are in the video?
Go over all frames and understand what is the average number of people.

Choose representative middle frame from which to identify persons accross frames.

In [27]:
def how_many_persons_video(data, person_thresh=0): # TODO: need to figure out person configuration score threshold
    no_persons = np.zeros(len(data))
    for i, frame_data in enumerate(data):
        subset = np.array(frame_data['subset'])
        for person in subset:
            if person[18] > person_thresh:
                no_persons[i] += 1
    return no_persons

In [25]:
# test
how_many_persons_video(data, 0)

array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 4., 4., 4., 3., 3., 3., 3., 3., 3., 4.,
       3., 3., 3., 3., 3., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])

## frame level

In [30]:
all_peaks = np.array(data[0]['all_peaks'])
subset = np.array(data[0]['subset'])
candidate = np.array(data[0]['candidate'])
person_thresh = 0 
joint_thresh = 0

def get_person_pose_frame(person, candidate, person_thresh=0, joint_thresh=0):
    ''' extracts body positions for given person (item in the "subset" list)'''
    confidence = person[18]     # check confidence level
    if confidence < person_thresh:
        return False
    joints = person[0:18]
    joints_locs = np.zeros((18, 2))
    for i, joint in enumerate(joints):
        j = int(joint)
        if joint == -1:
            joints_locs[i, :] = [-1, -1]
        elif candidate[j, 2] < joint_thresh:
            joints_locs[i, :] = [-1, -1]
        else:
            joints_locs[i, :] = candidate[j, 0:2]
            
    return joints_locs

# check
for person in subset:
    joints_locs = get_person_pose_frame(person, candidate)

In [31]:
joints_locs[joints_locs <= 0] = np.inf
joints_locs

array([[181.,  96.],
       [169.,  99.],
       [163., 100.],
       [158., 120.],
       [178., 115.],
       [177.,  99.],
       [ inf,  inf],
       [ inf,  inf],
       [167., 134.],
       [167., 157.],
       [165., 180.],
       [177., 133.],
       [175., 155.],
       [173., 173.],
       [179.,  93.],
       [184.,  94.],
       [174.,  90.],
       [ inf,  inf]])

## Matching between persons in different frames
So run previous function on all frames, see when getting head locations, and them match them based on minimal head loc change distanvce

In [ ]:
form sklearn.metrics import pairwise_distances
# get max characters in video
n_joints = 18
head_loc_idx = 1
max_no_people = max(how_many_persons_video(data))
n_frames = len(data)
joints_array = np.zeros((n_joints, 2, max_no_people, n_frames))

for i, frame_data in enumerate(data):
    subset = np.array(frame_data['subset'])

    if i == 0:
        for p, person in enumerate(subset):
            joints_locs = get_person_pose_frame(person, candidate)
            joints_array[:, :, p, i] = joints_locs
            
    else:
        for p, person in enumerate(subset):
            joints_locs = get_person_pose_frame(person, candidate)
            head_loc = joints_loc[head_loc_idx, :]
            dist_mat = pairwise_distances(head_loc, joints_array[head_loc_idx, :, :, i-1], 
                                          metric='euclidean', n_jobs=-1)
            min_idx = np.argmin(dist_mat)
            joints_array[:, :, p, i] = joints_locs
            

array([1., 0.])

# create bounding box for head location per person to be used in SORT tracking algorithm

In [52]:
def get_person_pose_frame(person, candidate, person_thresh=0, joint_thresh=0):
    ''' extracts body positions for given person (item in the "subset" list) '''
    no_joints = person[19]
    config_score = person[18]     # check configuration score
    if config_score < person_thresh:
        return False
    joints = person[0:18]
    joints_locs = np.zeros((18, 2))
    for i, joint in enumerate(joints):
        j = int(joint)
        if joint == -1 or j >= len(candidate):
            joints_locs[i, :] = [-1, -1]
        elif candidate[j, 2] < joint_thresh:
            joints_locs[i, :] = [-1, -1]
        else:
            joints_locs[i, :] = candidate[j, 0:2]
            
    return joints_locs, config_score, no_joints

In [53]:
def get_bbox_from_pose(joints_locs):
    joints_locs_min = joints_locs.copy()
    joints_locs_min[joints_locs <= 0] = np.inf
    
    minx = min(joints_locs_min[:, 0])
    miny = min(joints_locs_min[:, 1])
    maxx = max(joints_locs[:, 0])
    maxy = max(joints_locs[:, 1])
    return [minx, miny, maxx, maxy]

In [57]:
def get_all_joints(data):
    '''Function outputs an array of detected persons in each frame , 
    their joint body parts x, y locations and bounding box [x1,y1,x2,y2]'''
    joints_array = []
    for i, frame_data in enumerate(data):
        # print(f'Processing frame {i}')
        subset = np.array(frame_data['subset'])
        for person in subset:
            joints_locs, confidence, no_joints = get_person_pose_frame(person, candidate)
            bboxes = get_bbox_from_pose(joints_locs)
            joints_array.append([i] + list(bboxes) + list(joints_locs.flatten())+ [confidence, no_joints])
    return np.array(joints_array)

In [58]:
# test
joints_array = get_all_joints(data)

In [81]:
from sort.sort import *

#create instance of SORT
mot_tracker = Sort() 

# get detections
joints_array = get_all_joints(data)
object_ids = []
for i in range(len(data)):
    detections = joints_array[joints_array[:, 0] == i, 1:5]
    # update SORT
    # trackers is a np array where each row contains a valid bounding box and track_id (last column)
    print(detections.shape)
    trackers = mot_tracker.update(detections)
    print(trackers.shape)
    print('-----------------------------')
    object_ids += list(trackers[:, -1])
    
# concatenate object ids as new column to joints_array
object_ids = np.array(object_ids) - min(object_ids) # reindex from 0
print(object_ids.shape)
print(joints_array.shape)

(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(2, 5)
-----------------------------
(3, 4)
(2, 5)
-----------------------------
(3, 4)
(2, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
-----------------------------
(3, 4)
(3, 5)
------------------

In [74]:
joints_array = np.column_stack((joints_array, object_ids))
print(joints_array)

[42.0, 41.0, 40.0, 42.0, 41.0, 40.0, 42.0, 41.0, 40.0, 41.0, 40.0, 41.0, 40.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 44.0, 43.0, 41.0, 43.0, 41.0, 43.0, 41.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 41.0, 40.0, 41.0, 40.0, 41.0, 40.0, 41.0, 40.0, 43.0, 41.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0, 43.0, 40.0]


ValueError: all the input array dimensions except for the concatenation axis must match exactly

<Figure size 432x288 with 0 Axes>

In [71]:
min(object_ids)

22.0